# Data Augmentation, Batch Normalisation and 3 Layers

Using Data Augmentation to synthesise more samples, batch normalisation because its more optimal and 3 convolution layers because John said to use less

## Declare model 

In [1]:
import os, shutil, PIL
original_dataset_dir = '../data/primary_small/'
train_dir = os.path.join(original_dataset_dir, 'train')
validation_dir = os.path.join(original_dataset_dir, 'valid')
test_dir = os.path.join(original_dataset_dir, 'test')

from keras import layers
from keras import models
from keras import optimizers


model = models.Sequential()

model.add(layers.Conv2D(32,(3,3), activation='relu',
                       input_shape=(150,150,3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(7, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

model.summary()

Using TensorFlow backend.
/media/angus/Shared/Documents/Uni/final_year_project/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/media/angus/Shared/Documents/Uni/final_year_project/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/media/angus/Shared/Documents/Uni/final_year_project/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 148, 148, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 128)       36992     
_________________________________________________________________
batch_normalization_2 (Batch (None, 72, 72, 128)       512       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)     

## Data generation

In [17]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
         train_dir,
         target_size=(150,150),
         batch_size=32,
         class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150,150),
        batch_size=32,
        class_mode='categorical')

Found 5600 images belonging to 7 classes.
Found 700 images belonging to 7 classes.


## Training

In [18]:
history = model.fit_generator(
     train_generator,
     steps_per_epoch=100,
     epochs=30,
     validation_data=validation_generator,
     validation_steps=50)

Epoch 1/30
100/100 [==============================] - 336s 3s/step - loss: 1.3919 - acc: 0.4831 - val_loss: 6.8664 - val_acc: 0.1464
Epoch 2/30
100/100 [==============================] - 335s 3s/step - loss: 1.3675 - acc: 0.4759 - val_loss: 4.9840 - val_acc: 0.1401
Epoch 3/30
100/100 [==============================] - 335s 3s/step - loss: 1.3708 - acc: 0.4834 - val_loss: 6.6301 - val_acc: 0.1438
Epoch 4/30
100/100 [==============================] - 332s 3s/step - loss: 1.3639 - acc: 0.4913 - val_loss: 10.0238 - val_acc: 0.1461
Epoch 5/30
100/100 [==============================] - 352s 4s/step - loss: 1.3418 - acc: 0.4913 - val_loss: 4.6892 - val_acc: 0.1187
Epoch 6/30
100/100 [==============================] - 353s 4s/step - loss: 1.3297 - acc: 0.5056 - val_loss: 3.8748 - val_acc: 0.3712
Epoch 7/30
100/100 [==============================] - 330s 3s/step - loss: 1.3484 - acc: 0.4816 - val_loss: 3.4176 - val_acc: 0.4121
Epoch 8/30
100/100 [==============================] - 328s 3s/step -

In [4]:
model.save('./../models/seti_small_7.h5')

## Evaluate on un-augmented data

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

unAug_train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150,150),
        batch_size=20,
        class_mode='categorical')
unAug_validation_generator = test_datagen.flow_from_directory(
            validation_dir,
            target_size=(150,150),
            batch_size=20,
            class_mode='categorical')

Found 5600 images belonging to 7 classes.
Found 700 images belonging to 7 classes.


In [14]:
model.fit_generator(generator=unAug_train_generator,steps_per_epoch=None, epochs=1,verbose=1,validation_data=unAug_validation_generator)

Epoch 1/1
280/280 [==============================] - 547s 2s/step - loss: 1.3530 - acc: 0.4959 - val_loss: 10.7980 - val_acc: 0.1429


# The problem

data augmentation has led to worse accuracy, but I seem to have left out rescale for the training generator, either by mistake or for reasons I have forgotten. model 6 is the first version, model 7 is the version with the right rescaling